In [ ]:
from __future__ import division, print_function

# Third-party
import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
plt.style.use('apw-notebook')
%matplotlib inline
import h5py

# Custom
import gala.coordinates as gc
import gala.dynamics as gd
import gala.integrate as gi
import gala.potential as gp
from gala.units import galactic

In [ ]:
all_streams = []
all_weights = []
with h5py.File("../cache/sph_iso.hdf5", "r") as f:    
    for key in f['mock_streams']:
        d_pos = f['mock_streams'][key]["stream_pos"]
        pos = d_pos[:] * u.Unit(d_pos.attrs['unit'])
        
        if pos.shape[1] > 0: # and np.max(np.sqrt(np.sum(pos**2, axis=0))) > 10*u.kpc:
            d_vel = f['mock_streams'][key]["stream_vel"]
            vel = d_vel[:] * u.Unit(d_vel.attrs['unit'])
            all_streams.append(gd.CartesianPhaseSpacePosition(pos=pos, vel=vel))
            all_weights.append(f['mock_streams'][key]['stream_weights'][:].sum())
            
        if len(all_streams) == 2048:
            break
            
len(all_streams)

In [ ]:
fig,axes = plt.subplots(2, 2, figsize=(10,10), sharex=True, sharey=True)

style = dict(linestyle='none', alpha=0.1, color='k', marker=',')

for stream in all_streams:
    axes[0,0].plot(stream.pos[0], stream.pos[1], **style)
    axes[1,0].plot(stream.pos[0], stream.pos[2], **style)
    axes[1,1].plot(stream.pos[1], stream.pos[2], **style)

axes[0,0].set_xlim(-45, 45)
axes[0,0].set_ylim(-45, 45)

axes[0,0].set_ylabel('$y$ [kpc]')
axes[1,0].set_ylabel('$z$ [kpc]')
axes[1,0].set_xlabel('$x$ [kpc]')
axes[1,1].set_xlabel('$y$ [kpc]')

fig.tight_layout()

axes[0,1].set_visible(False)

Plot a mass profile

In [ ]:
bins = np.logspace(-1., np.log10(300), 33) # 0.1 to 300 kpc
all_H = np.zeros(bins.size-1)

with h5py.File("../cache/sph_iso.hdf5", "r") as f:
    count = 0
    for key in f['mock_streams']:
        if int(key) % 1000 == 0:
            print(key)
        count += 1
        pos = f['mock_streams'][key]["stream_pos"][:]
        if pos.shape[1] > 0:
            r = np.sqrt(np.sum(pos**2, axis=0))
            w = f['mock_streams'][key]["stream_weights"][:]
            H,_ = np.histogram(r, bins=bins, weights=w)
            all_H += H

V = 4/3*np.pi*(bins[1:]**3 - bins[:-1]**3)
bin_cen = (bins[1:]+bins[:-1])/2.

In [ ]:
count

In [ ]:
fig,axes = plt.subplots(1, 2, figsize=(12,6), sharex=True)

style = dict(ls='-', marker=None, lw=1.5)
axes[0].loglog(bin_cen, all_H/V, **style)
axes[1].loglog(bin_cen, np.cumsum(all_H), **style)

axes[0].set_xlim(0.1, 350)

axes[0].set_xlabel('$r$ [kpc]')
axes[1].set_xlabel('$r$ [kpc]')

axes[0].set_ylabel(r'$\rho(r)$ [kpc]')
axes[1].set_ylabel(r'$M(<r)$ [kpc]')

fig.tight_layout()

## Projected surface brightness

In [ ]:
bins = np.linspace(-250, 250, 512+1)
H = np.zeros((bins.size-1, bins.size-1))

with h5py.File("../cache/sph_iso.hdf5", "r") as f:
    for key in f['mock_streams']:
        if int(key) % 1000 == 0:
            print(key)
            
        pos = f['mock_streams'][key]["stream_pos"][:]
        if pos.shape[1] > 0:
            w = f['mock_streams'][key]["stream_weights"][:]
            H += np.histogram2d(pos[0], pos[1], bins=(bins,bins), weights=w)[0]

In [ ]:
X,Y = np.meshgrid(bins, bins)

plt.figure(figsize=(10,10))
plt.pcolormesh(X, Y, H+1E-7, norm=col.LogNorm(),
               cmap='hesperia', vmin=1E0, vmax=1E5)

# Pretty

In [ ]:
import matplotlib.colors as col
import matplotlib as mpl

In [ ]:
cmap = plt.get_cmap('laguna')
# cmap = plt.get_cmap('viridis')

In [ ]:
norm = col.LogNorm(vmin=5E3, vmax=1E5)

In [ ]:
plt.hist(all_weights, bins=np.logspace(3,7,16))
plt.xscale('log')

In [ ]:
fig,ax = plt.subplots(1, 1, figsize=(10,10))

style = dict(linestyle='none', alpha=0.25, marker=',')

for stream,w in zip(all_streams,all_weights):
    ax.plot(stream.pos[0], stream.pos[1], color=cmap(norm(w)), **style)

ax.set_xlim(-45, 45)
ax.set_ylim(-45, 45)

fig.tight_layout()

ax.xaxis.set_visible(False)
ax.yaxis.set_visible(False)
fig.set_facecolor('k')
ax.set_axis_bgcolor('k')